In [ ]:
!pip install -U transformers tokenizers

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.11.3)


Requirement already up-to-date: tokenizers in /usr/local/lib/python3.6/dist-packages (0.10.3)


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizerTUR = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
print(f"VOC size is: {tokenizerTUR.vocab_size}")
print(f"The model is: {type(tokenizerTUR)}")

VOC size is: 32000
The model is: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [ ]:

tokenizerEN = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"VOC size is: {tokenizerEN.vocab_size}")
print(f"The model is: {type(tokenizerEN)}")

VOC size is: 30522
The model is: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [ ]:
word_en = "telecommunication"
print(f"is in Turkish Model ? {word_en in tokenizerTUR.vocab}")
print(f"is in English Model ? {word_en in tokenizerEN.vocab}")

is in Turkish Model ? False
is in English Model ? True


In [ ]:
tokens = tokenizerTUR.tokenize(word_en)
tokens

['tel', '##eco', '##mm', '##un', '##ica', '##tion']

Unknown words are broken down into parts. The parts are tokens known to the model (in the vocab)

The English model does know about the word, so it does not need to break it into parts

In [ ]:
tokenizerEN.tokenize(word_en)

['telecommunication']

In [ ]:
long_word_tur = "Muvaffakiyetsizleştiricileştiriveremeyebileceklerimizdenmişsinizcesine"
tokenizerTUR.tokenize(long_word_tur)

['muvaffak',
 '##iyet',
 '##siz',
 '##les',
 '##tir',
 '##ici',
 '##les',
 '##tir',
 '##iver',
 '##emeye',
 '##bilecekleri',
 '##mi',
 '##z',
 '##den',
 '##mis',
 '##siniz',
 '##cesine']

This is the WordPiece Algorithmn. BERT, DistilBERT, and ELECTRA require a WordPiece tokenizer


In [ ]:
!pip install nltk

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import nltk
from nltk.corpus import gutenberg
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!ls /root/nltk_data/corpora/gutenberg

README		       burgess-busterbrown.txt	milton-paradise.txt
austen-emma.txt        carroll-alice.txt	shakespeare-caesar.txt
austen-persuasion.txt  chesterton-ball.txt	shakespeare-hamlet.txt
austen-sense.txt       chesterton-brown.txt	shakespeare-macbeth.txt
bible-kjv.txt	       chesterton-thursday.txt	whitman-leaves.txt
blake-poems.txt        edgeworth-parents.txt
bryant-stories.txt     melville-moby_dick.txt


In [ ]:
plays = ['shakespeare-macbeth.txt', 'shakespeare-hamlet.txt', 'shakespeare-caesar.txt']
shakespeare = [" ".join(s) for ply in plays for s in gutenberg.sents(ply)]

In [ ]:
from tokenizers.processors import TemplateProcessing

special_tokens = ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
temp_proc = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", special_tokens.index("[CLS]")),
        ("[SEP]", special_tokens.index("[SEP]")),
    ],
)

In [ ]:
from tokenizers import Tokenizer
from tokenizers.normalizers import (Sequence, Lowercase, NFD, StripAccents)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.decoders import BPEDecoder

In [ ]:
tokenizer = Tokenizer(BPE())

Pre-processing:
1. Normalizer
2. Pre-tokenizer

In [ ]:
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = BPEDecoder()
tokenizer.post_processor = temp_proc

In [ ]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(vocab_size=5000,
    special_tokens=special_tokens)

tokenizer.train_from_iterator(shakespeare, trainer)
print(f"Trained vocab size: {tokenizer.get_vocab_size()}")

Trained vocab size: 5000


In [ ]:
sen = "Is this a dagger which I see before me, the handle toward my hand?"
sen_enc = tokenizer.encode(sen)
print(f"Output: {format(sen_enc.tokens)}")

Output: ['[CLS]', 'is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', 'le', 'toward', 'my', 'hand', '?', '[SEP]']


In [ ]:
sen_enc2 = tokenizer.encode("Macbeth and Hugging Face")
print(f"Output: {format(sen_enc2.tokens)}")

Output: ['[CLS]', 'macbeth', 'and', 'hu', 'gg', 'ing', 'face', '[SEP]']


In [ ]:
two_enc = tokenizer.encode("I like Hugginf Face!", "He likes Macbeth!")
print(f"Output: {format(two_enc.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'in', 'f', 'face', '!', '[SEP]', 'he', 'likes', 'macbeth', '!', '[SEP]']


In [ ]:
tokenizer.model.save('.')

['./vocab.json', './merges.txt']

In [ ]:
!wc -l ./merges.txt

4948 ./merges.txt


In [ ]:
!head -6 ./merges.txt

#version: 0.2 - Trained by `huggingface/tokenizers`
t h
o u
a n
th e
r e


In [ ]:
# save entire pipeline
tokenizer.save("MyBPETokenizer.json")
tokenizerFromFile = Tokenizer.from_file("MyBPETokenizer.json")
sen_enc3 = tokenizerFromFile.encode("I still like Hugging Face and Macbeth")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['[CLS]', 'i', 'still', 'like', 'hu', 'gg', 'ing', 'face', 'and', 'macbeth', '[SEP]']


In [ ]:
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.normalizers import BertNormalizer

In [ ]:
tokenizer = Tokenizer(WordPiece())
tokenizer.normalizer = BertNormalizer()
tokenizer.pre_torkinzer = Whitespace()
tokenizer.decoder = WordPieceDecoder()

In [ ]:
help(tokenizer)

Help on Tokenizer object:

class Tokenizer(builtins.object)
 |  A :obj:`Tokenizer` works as a pipeline. It processes some raw text as input
 |  and outputs an :class:`~tokenizers.Encoding`.
 |  
 |  Args:
 |      model (:class:`~tokenizers.models.Model`):
 |          The core algorithm that this :obj:`Tokenizer` should be using.
 |  
 |  Methods defined here:
 |  
 |  __getnewargs__(...)
 |  
 |  __getstate__(...)
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __setstate__(...)
 |  
 |  add_special_tokens(self, tokens)
 |      Add the given special tokens to the Tokenizer.
 |      
 |      If these tokens are already part of the vocabulary, it just let the Tokenizer know about
 |      them. If they don't exist, the Tokenizer creates them, giving them a new id.
 |      
 |      These special tokens will never be processed by the model (ie won't be split into
 |      multiple tokens), and they 

In [ ]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(vocab_size=5000,
  special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tokenizer.train_from_iterator(shakespeare, trainer=trainer)
output = tokenizer.encode(sen)
print(output.tokens)

['is ', '##this ', '##a ', '##dagg', '##er ', '##which ', '##i see ', '##before ', '##me', '##, the ', '##hand', '##le ', '##toward ', '##my ', '##hand', '##?']


In [ ]:
tokenizer.decode(output.ids)

'is this a dagger which i see before me, the handle toward my hand?'

In [ ]:
# for an example of unknown tokens, for some reason this isn't wordpiece

tokenizer.encode("Kralsin aslansin Macbeth!").tokens

['k',
 '##ra',
 '##l',
 '##sin',
 '## ',
 '##as',
 '##lan',
 '##sin',
 '## ',
 '##macbeth',
 '##!']

In [ ]:
# pre-made, but untrained tokeinzers

from tokenizers import (ByteLevelBPETokenizer,
    CharBPETokenizer,
    SentencePieceBPETokenizer,
    BertWordPieceTokenizer)